# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
# Image Pre Processing
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
# Apply geometric trasnformations - image augmentation
# Transform the images so that our model doesn't overlearn
train_datagen = ImageDataGenerator(
    # Apply feature scalling in each pixel like normalization
    # It's compulsory
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Loading train images
training_set = train_datagen.flow_from_directory(
    'dataset/training_set', # directory
    # target_size=(150, 150),
    target_size=(64, 64), # Smaller so don't take too long
    batch_size=32, # default 32
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
# We keep the images like if was the original ones
# We just use the recal  to normalize it
# and to not overlearn it
test_datagen = ImageDataGenerator(rescale=1./255)

# Loading test images
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
# Initialize module
cnn = tf.keras.models.Sequential()

2021-12-23 20:22:45.024234: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Step 1 - Convolution

In [6]:
# Convolutional layer
cnn.add(
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=3, # Dimensions of the filter map
        activation='relu', # Always relu on first layers
        input_shape=[64, 64, 3] # 3 for RGB, 1 to black/white
    )
)

### Step 2 - Pooling

In [7]:
# Pooling layer
cnn.add(
    tf.keras.layers.MaxPool2D(
        pool_size=2, # (2X2) Size of the pooled feature map on the feature map
        strides=2 # Shift frame every 2 pixels
    )
)

### Adding a second convolutional layer

In [8]:
# Convolutional layer
cnn.add(
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=3, # Dimensions of the filter map
        activation='relu', # Always relu on first layers
        # Input shape is only on the first layer
    )
)

# Pooling layer
cnn.add(
    tf.keras.layers.MaxPool2D(
        pool_size=2, # (2X2) Size of the pooled feature map on the feature map
        strides=2 # Shift frame every 2 pixels
    )
)

### Step 3 - Flattening

In [9]:
# Add flattening layer
cnn.add(
    tf.keras.layers.Flatten()
)

### Step 4 - Full Connection

In [10]:
# Add Full Connection layer
cnn.add(
    tf.keras.layers.Dense(
        # Number of hidden neurons
        units=128, # We need a bigger number because computation is more complex
        activation='relu'
    )
)

### Step 5 - Output Layer

In [11]:
# Add Output layer
cnn.add(
    tf.keras.layers.Dense(
        # Number of hidden neurons
        units=1, # Binary Classification - Cat or Dog
        # Otherwise, multiclass classification we have softmax
        activation='sigmoid'
    )
)

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(
    x = training_set,
    validation_data = test_set, # Test set
    epochs = 25 # Number of epochs
)

2021-12-23 20:22:59.737448: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
250/250 [==============================] - 55s 216ms/step - loss: 0.6584 - accuracy: 0.5929 - val_loss: 0.5956 - val_accuracy: 0.6865
Epoch 2/25
250/250 [==============================] - 53s 212ms/step - loss: 0.5824 - accuracy: 0.6949 - val_loss: 0.5949 - val_accuracy: 0.6790
Epoch 3/25
250/250 [==============================] - 55s 220ms/step - loss: 0.5496 - accuracy: 0.7153 - val_loss: 0.5157 - val_accuracy: 0.7465
Epoch 4/25
250/250 [==============================] - 53s 212ms/step - loss: 0.5239 - accuracy: 0.7345 - val_loss: 0.5159 - val_accuracy: 0.7375
Epoch 5/25
250/250 [==============================] - 52s 209ms/step - loss: 0.4969 - accuracy: 0.7611 - val_loss: 0.4936 - val_accuracy: 0.7695
Epoch 6/25
250/250 [==============================] - 54s 216ms/step - loss: 0.4885 - accuracy: 0.7614 - val_loss: 0.4897 - val_accuracy: 0.7655
Epoch 7/25
250/250 [==============================] - 52s 207ms/step - loss: 0.4649 - accuracy: 0.7755 - val_loss: 0.4664 - val_ac

## Part 4 - Making a single prediction

In [22]:
import numpy as np
from keras.preprocessing import image

# Load single image to deploy to our production model in
# format of PIL
prod_image = image.load_img(
    'dataset/single_prediction/cat_or_dog_3.jpeg',
    # Compulsory - same size as the training
    target_size = (64, 64)
)

# Convert the PIL image format into an array
prod_image = image.img_to_array(
    prod_image
) # converts it to numpy array

# Predict as to be called in the same format of the training
# we need to create batches of images
prod_image = np.expand_dims(
    prod_image,
    axis = 0 # Dimension of the batch on first dimension
)

# Prediction, normalizing the result
result = cnn.predict(
    prod_image/255.0
)

# Show what each number is
training_set.class_indices

# Treat the result output - access the batch and then the single element of the batch
if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'

In [23]:
print(prediction)

dog
